In [44]:
import openpyxl
p=[i for i in range(1,38)]
n=[i for i in range(1,38)]
P=[i for i in range(37)]
N=[i for i in range(37)]

book1 = openpyxl.load_workbook("Block location (5).xlsx")
sh=book1["NC2NC"]

#Potential facility centres among the cities
Potential_national_facility=[]
for i in p:
    Potential_national_facility.append(sh.cell(0+1,i+1).value)

#incorporating the distance matrix of National centres
dn2=[]
dn2n=[]
for i in p:
    for j in n:
        dn2.append(int(sh.cell(i+1,j+1).value))
    dn2n.append(dn2)
    dn2=[]
    
#incorporating the demand of the States    
Dn2n=[]
ND=book1["NC Population"] #NC Demand
for i in Potential_national_facility:
    for j in range(1,38):
        if i==str(ND.cell(j+1,0+1).value):
            Dn2n.append(int(ND.cell(j+1,2+1).value))

# facility location problem to select national centres

In [45]:
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
from itertools import product

m=gp.Model('Facility_Centre')
y=m.addVars(P,N,vtype=GRB.BINARY,name='Y')
x=m.addVars(P,vtype=GRB.BINARY,name='X')

m.addConstrs((gp.quicksum(y[i,j] for i in P)==1 for j in N));
m.addConstr((gp.quicksum(x[i] for i in P)==5));
m.addConstrs((y[i,j] <= x[i] for j in N for i in P))

vj=[45104, 11275, 1556487, 197530, 46000, 1556487, 13088, 13088, 13088, 768118, 27813, 18542, 22662, 33993, 235042, 737107, 55625, 384842, 28051, 17446, 425766, 5563, 9845, 157612, 9271, 1056513, 1070383, 6400, 48816, 17446, 34766, 327676, 60500, 14278, 695592, 25495, 211011]

m.setObjective(gp.quicksum(dn2n[i][j] * Dn2n[j]/vj[j] * y[i,j] for j in N for i in P ),GRB.MINIMIZE)

m.Params.Cuts=0
m.Params.Heuristics=0
m.Params.Presolve=0
m.Params.Symmetry=0
m.Params.RINS=0
m.Params.Aggregate=0
m.Params.Disconnected=0
m.optimize()

m.printAttr('X')
TNC=[]
for i in P:
    if x[i].X==1:
        TNC.append(Potential_national_facility[i])
    else:
        continue
print(TNC)
Arcs=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            Arcs.append(Potential_national_facility[i]+" : "+Potential_national_facility[j])


Arcs1={}
list5=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            list5.append(Potential_national_facility[j])
    if len(list5)!=0:
        Arcs1[str(Potential_national_facility[i])]=list5
        list5=[]
    else:
        continue

#Distance from transshipment National Centres to State Capitals
dtn2n=[]
for i in TNC:
    if i in Potential_national_facility:
        dtn2n.append(dn2n[Potential_national_facility.index(i)])


Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Presolve to value 0
Set parameter Symmetry to value 0
Set parameter RINS to value 0
Set parameter Aggregate to value 0
Set parameter Disconnected to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1407 rows, 1406 columns and 4144 nonzeros
Model fingerprint: 0x966e2598
Variable types: 0 continuous, 1406 integer (1406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+02, 7e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Variable types: 0 continuous, 1406 integer (1406 binary)

Root relaxation: objective 2.054159e+06, 250 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds   

In [46]:
Manufacturers=['Serum Institute of India, Pune','Bharat Biotech International Ltd, Hyderabad'] #,'Cadila Healthcare Ltd, Ahmedabad','Trialed and manufactured in India by Dr. Reddy Lab.','Serum Institute of India, Pune']#,'Biological E Ltd, Hyderabad','Genova, Pune','Bharat Biotech International Ltd, Hyderabad','Aurobindo Pharma Ltd, Hyderabad']
Suppliers=['A','B'] #,'C','D']
V=[300, 25, 50]
PCapr=[475000,15000,5]

Wijrm2n=[]
Wij=[]
wi=[]
NDA=book1["M2NAir"] #Accessing the data of manufacturers to Distance from excel sheet 
NDR=book1["M2NRoad"]
NDM=book1["M2NMotor"]
for i in range(1,len(Potential_national_facility)+1):
    for j in range(1,len(Potential_national_facility)+1):
        if (NDA.cell(i+1,0+1).value in Manufacturers or NDA.cell(i+1,0+1).value in Suppliers) and NDA.cell(0+1,j+1).value in TNC:
            wi.append(int(NDA.cell(i+1,j+1).value))
            wi.append(int(NDR.cell(i+1,j+1).value))
            wi.append(int(NDM.cell(i+1,j+1).value))
            if len(wi)!=0:
                Wij.append(wi)
                wi=[]
            else:
                print(wi)
    if len(Wij)!=0:        
        Wijrm2n.append(Wij)
        Wij=[]
    else:
        continue

dM2=[]
dM2T=[]
dM2TN=[]
M2NA=book1["M2TNCA"] #Accessing the data of manufacturers to Distance from excel sheet 
M2N=book1["M2NC"]
for i in range(50):
    for j in range(len(Potential_national_facility)+1):
        if (M2NA.cell(i+1,0+1).value in Manufacturers or M2NA.cell(i+1,0+1).value in Suppliers) and M2NA.cell(0+1,j+1).value in TNC:
            dM2.append(int(M2NA.cell(i+1,j+1).value))
            dM2.append(int(M2N.cell(i+1,j+1).value))
            dM2.append(int(M2N.cell(i+1,j+1).value))
            if len(dM2)!=0:
                dM2T.append(dM2)
                dM2=[]
    if len(dM2T)!=0:        
        dM2TN.append(dM2T)
        dM2T=[]
    else:
        continue

Dm2={}
list2=[]
for i in P:
    for j in N:
        if y[i,j].X == 1:
            list2.append(Dn2n[j])
    if len(list2)!=0:
        Dm2[str(Potential_national_facility[i])]=list2
        list2=[]
    else:
        continue

Dm2n={}
for i in Dm2:
    Dm2n[i]=sum(Dm2[i])

DM2N=[]
for i in Dm2n:
    DM2N.append(Dm2n[i])

In [47]:
DM2NP=[]
DM2NP.append(DM2N)
VR=0.3

In [48]:
M=[i for i in range(len(Manufacturers))]
P=[i for i in range(5)]
for p in P:
    if p>(0):
        DM2NP.append([int(i*(1-1/(p+1))) for i in DM2N])
DM2NP

[[254000136, 271572988, 168978824, 470760837, 45256788],
 [127000068, 135786494, 84489412, 235380418, 22628394],
 [169333424, 181048658, 112652549, 313840558, 30171192],
 [190500102, 203679741, 126734118, 353070627, 33942591],
 [203200108, 217258390, 135183059, 376608669, 36205430]]

## Set of index
\begin{align*}
M &: \text{Set of vaccine manufacturers } = {1,2,\dots,|M|} \\
S &: \text{Set of Exporters from where the user imports vaccines } = {|M|+1,|M|+2,\dots,|M|+|S|} \\
N &: \text{Set of Demand nodes } = {1,2,\dots,|N|} \\
T &: \text{Set of Vaccine types } = {1,2,\dots,|T|} \\
P &: \text{Set of periods in the planning horizon } = {0,1,\dots,|P|} \\
R &: \text{Set of MRUs across all modes of transportation } = {1,2,\dots,|R|} \\
F_j &: \text{Set of candidate refrigerators at location } j
\end{align*}

## Parameters
\begin{align*}
d_{ij}^r &: \text{Distance between location } i \text{ and } j \text{ using transportation mode } r \text{ (km)} \\
C\_Trans_r &: \text{Cost of transportation per km with MRU } r^{th} \text{ (\$)} \\
h_j &: \text{Cost of holding one unit of vaccine in location } j \\
C\_Hir_r^p &: \text{Cost of hiring vehicle } r \text{ in period } p \\
Cap\_R_j^f &: \text{Capacity of } f^{th} \text{ refrigerator at location } j. \\
M\_Cap_i^{tp} &: \text{Manufacturing capacity of vaccine type } t \text{ at } i^{th}  \text{manufacturer in period } p \\
M\_Exp_i^{tp} &: \text{Exporting capacity of vaccine type } t \text{ at } i^{th}  \text{manufacturer in period } p \\
S\_Cap_i^{tp} &: \text{Availability of vaccine type } t \text{ at importing node } i \text{ in period } p \\
P\_Cap_r &: \text{Capacity of } r^{th} \text{MRU.} \\
C\_Ref_j^f &: \text{Cost of opening and maintaining the} f^{th} \text{ refrigerator at location } j \\
D_j &: \text{Demand at location } j \text{ before the beginning of vaccination drive} \\
v &: \text{Vaccine (litre) in one shot of vaccine.} \\
Vac_j^{tp} &: \text{Number of vaccine type } t \text{ available at location } j \text{ in period } p \\
pd_j &: \text{Average vaccine disbursement rate at location } j \\
Vac\_R_j^{tpz} &: \text{Number of type } t \text{ vaccines for } z^{th} \text {re-vaccination available at location } j \text{ in period } p \\
rv_z &: \text{Period at which } z^{th} \text{ re-vaccination starts } \\
D_j^p &: \text{Demand at location } j  \text{ in the period  } p\\
D\_var_j^{tpz} &: \text{Variable demand of vaccine type } t \text{ at location  } j \text{ in period } p \text{ for } z^{th} \text{ re-vaccination} \\
\pi\_rg_p &: \text{The fraction of fixed demand to receive vaccines in period } p  \\
\pi\_rv_p &: \text{The fraction of variable demand to receive vaccines in period } p\\
rv\_rate_j^z &: \text{re-vaccination rate for the } z^{th} \text{re-vaccination} \\
f_1 &: \text{Fraction of total vaccine shots dedicated for regular  v\ accination in period} p \\
f_2 &: \text{Fraction of total vaccine shots dedicated for r e-\ vaccination in period} p \\
\end{align*}
 

$ \text{Where } z\in Z \text{ is numbers of re-vaccination (doses of re-vaccination per individual).} $


In [49]:
# Define ranges and sets
M = [i for i in range(len(Manufacturers))]
S = [i for i in range(len(M), len(Manufacturers) + len(Suppliers))]
N = [i for i in range(len(TNC))]
T = M + S
TD = 0
R = [0, 1, 2]
F = [0, 1, 2] 

# capacities and costs
CapR = [400000000, 200000000, 10000000]
CF = [[158191, 15000, 5000] for j in N]

# Initialize matrices to zero and assign values conditionally
MCap = [[[0 for t in T] for i in M + S] for p in P]
Mexp = [[[0 for t in T] for i in M + S] for p in P]
SCap = [[[0 for t in T] for i in M + S] for p in P]

for p in P:
    for i in M + S:
        for t in T:
            if i == t:
                if i < len(M):  # For manufacturers
                    MCap[p][i][t] = int(120000000)
                    Mexp[p][i][t] = int(2000000)
                else:  # For suppliers
                    SCap[p][i][t] = int(1000000)

# Check results for MCap of the first period
MCap[0]


[[120000000, 0, 0, 0], [0, 120000000, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [50]:
ex=3
rv=2
pd = 0.8
rpd=0.7
Z = 1

In [51]:
f1=0.3
f2=1-f1
DM2NPPP=[]
DM2NPPP.append(DM2N)
PctP=[0 for i in P]
PctP[0]=0.99999*((sum(sum(MCap[0][i][t] for t in T) for i in M)-sum(sum(Mexp[0][i][t] for t in T) for i in M)+sum(sum(SCap[0][i][t] for t in T) for i in S))/(sum(DM2NPPP[0])))
for p in range(1,len(P)):
    a=[]
    for j in N:
        a.append(int(DM2NPPP[p-1][j]*(1-PctP[p-1])))
    DM2NPPP.append(a)
    if p<rv:
        PctP[p]=0.99999*(((sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S))/(sum(DM2NPPP[p]))))
    else:
        PctP[p]=0.99999*((f1*(sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S))/(sum(DM2NPPP[p]))))
PctP

[0.19659970422864745,
 0.24470952508963476,
 0.09719817760906015,
 0.10766280665156337,
 0.12065260531544256]

# model

In [52]:
import math as ma

m = gp.Model('VDN')


XMijtp  = m.addVars(M,N,T,P, vtype=GRB.CONTINUOUS, name='XMijtp')
XIijtp  = m.addVars(S,N,T,P, vtype=GRB.CONTINUOUS, name='XIijtp')
xRMijtpz = m.addVars(M,N,T,P,range(1,Z+1), vtype=GRB.CONTINUOUS, name='xRMijtpz')
xRIijtpz = m.addVars(S,N,T,P,range(1,Z+1), vtype=GRB.CONTINUOUS, name='xRIijtpz')

TRijp=m.addVars(M+S,N,P, vtype=GRB.INTEGER, name='TRijp')
Yijrp=m.addVars(M+S,N,R,P, vtype=GRB.BINARY, name='Yijrp')
neta=m.addVars(N,F,vtype=GRB.BINARY, name='neta')


Vac_avai_jtp = gp.tupledict()

m.addConstrs(gp.quicksum(XMijtp[i,j,t,p] for j in N)+gp.quicksum(xRMijtpz[i,j,t,p,z] for j in N for z in range (1, Z+1))<=(MCap[p][i][t])-(Mexp[p][i][t]) for p in range(0,len(P)) for i in M for t in T)

m.addConstrs(gp.quicksum(XIijtp[i,j,t,p] for j in N)+gp.quicksum(xRIijtpz[i,j,t,p,z] for j in N for z in range(1, Z+1))<=SCap[p][i][t] for p in range(0,len(P)) for i in S for t in T)


m.addConstrs(Wijrm2n[i][j][r]-Yijrp[i,j,r,p]>=0 for i in M+S for j in N for r in R for p in range(0,len(P)))
m.addConstrs(gp.quicksum(Yijrp[i,j,r,p] for r in R)<=1 for i in M+S for j in N for p in range(0,len(P)))


DM2NP=[[0  for i in N] for p in P]
Cost=[5,0.54,0.23]
DZ=[254000136, 271572988, 168978824, 470760837, 45256788]
DZ=[254000136, 271572988, 156437522, 133630914, 337510504, 44876207, 12541302]


for p in range(0,min(len(P),rv)):
    for j in N:
        DM2NP[p][j]=DM2NPPP[p][j]

for j in N:
    for p in range(rv,len(P)):
        DM2NP[p][j] = DM2NP[p-1][j]-gp.quicksum(XMijtp[i,j,t,p-1] for i in M for t in T)-gp.quicksum(XIijtp[i,j,t,p-1] for i in S for t in T)


# Constraints with Perishability Factor
for j in N:
    for t in T:
        for p in P:
            if p < Z * rv:
                Vac_avai_jtp[j, t, p] = gp.quicksum(
                    ((1 - pd) ** (p - k)) * 
                    (gp.quicksum(XMijtp[i, j, t, k] for i in M) + gp.quicksum(XIijtp[i, j, t, k] for i in S))
                    for k in range(max(0, p - ex), p + 1)
                )
            if p >= Z * rv:
                Vac_avai_jtp[j, t, p] = gp.quicksum(
                    ((1 - pd) ** (p - k)) * 
                    (gp.quicksum(XMijtp[i, j, t, k] for i in M) + gp.quicksum(XIijtp[i, j, t, k] for i in S))
                    for k in range(max(0, p - ex), p + 1)
                ) + gp.quicksum(
                    gp.quicksum(
                        (1 - rpd) ** l *
                        (gp.quicksum(xRMijtpz[i, j, t, p - l, z] for i in M) + gp.quicksum(xRIijtpz[i, j, t, p - l, z] for i in S))
                        for l in range(min(p - z * rv + 1, ex + 1))
                    ) for z in range(1, Z + 1)
                )
         
m.addConstrs(
    gp.quicksum(Vac_avai_jtp[j, t, p] for t in T) <= gp.quicksum(CapR[f] * neta[j, f] for f in F)
    for j in N for p in P
)


D_v_jtpz = gp.tupledict()
for p in range(0,min(len(P),rv)):
    for j in N:
        for t in T:
            for z in range(1,Z+1):
                D_v_jtpz[j, t, p, z]=0

for p in range(rv, len(P)):
    for j in N:
        for t in T:
            for z in range(1, Z+1):                
                 D_v_jtpz[j, t, p, z] = pd*(gp.quicksum(((1-pd)**(p-rv-k))*(gp.quicksum(XMijtp[i,j,t,k] for i in M)
                                                                                           +gp.quicksum(XIijtp[i,j,t,k] for i in S)) for k in range (max(0,p-rv-ex), (p-rv)+1)))


m.addConstrs(gp.quicksum(XMijtp[i,j,t,p] for i in M for t in T) + gp.quicksum(XIijtp[i,j,t,p] for i in S for t in T)>=ma.ceil(PctP[p]*(DM2NPPP[p][j])) for p in range(0,len(P)) for j in N)

m.addConstrs(
    gp.quicksum(xRMijtpz[i, j, t, p, z] for i in M) + 
    gp.quicksum(xRIijtpz[i, j, t, p, z] for i in S) >= 
    0.07 * D_v_jtpz[j, t, p, z]
    for p in range(rv, len(P)) for j in N for t in T for z in range(1, Z + 1)
)


m.addConstrs(
    gp.quicksum(xRMijtpz[i, j, t, p, z] for i in M) + 
    gp.quicksum(xRIijtpz[i, j, t, p, z] for i in S) == 0 
    for p in range(0, min(len(P), rv)) for j in N for t in T for z in range(1,Z + 1)
)


# Ensure total vaccinations do not exceed capacity threshold
m.addConstrs(
    gp.quicksum(
        gp.quicksum(xRMijtpz[i, j, t, p, z] for i in M) + 
        gp.quicksum(xRIijtpz[i, j, t, p, z] for i in S)
        for t in T for z in range(1, Z + 1)
    ) <= ma.floor(
        0.9999999 * f2 * (
            sum(sum(MCap[p][i][t] for t in T) for i in M) - 
            sum(sum(Mexp[p][i][t] for t in T) for i in M) + 
            sum(sum(SCap[p][i][t] for t in T) for i in S)
        )
    )
    for p in range(rv, len(P)) for j in N
)

m.addConstrs(ma.ceil(0.9999999*(f2)*(sum(sum(MCap[p][i][t]for t in T) for i in M)-sum(sum(Mexp[p][i][t]for t in T) for i in M)+sum(sum(SCap[p][i][t]for t in T) for i in S)))>= gp.quicksum(D_v_jtpz[j,t,p, z] for t in T for z in range(1, Z+1) ) for p in range(rv, len(P)) for j in N)    



{(2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>}

In [53]:
# Adding the variable
wijrp = m.addVars(M + S, N, R, P, vtype=GRB.CONTINUOUS, name="wijrp")

# Corrected constraints
m.addConstrs(
    gp.quicksum(PCapr[r] * wijrp[i, j, r, p] for r in R) -
    gp.quicksum((XMijtp[i, j, t, p]) +(xRMijtpz[i, j, t, p, z]) for t in T for z in range(1, Z + 1)) * 0.005 >= 0
    for j in N for i in M for p in P
)

m.addConstrs(
    gp.quicksum(PCapr[r] * wijrp[i, j, r, p] for r in R) -
    gp.quicksum((XIijtp[i, j, t, p]) +(xRIijtpz[i, j, t, p, z]) for t in T for z in range(1, Z + 1)) * 0.005 >= 0
    for j in N for i in S for p in P
)

m.addConstrs(wijrp[i,j,r,p]>=TRijp[i,j,p]-200*(1-Yijrp[i,j,r,p]) for r in R for j in N for i in M+S for p in range(0,len(P)))
m.addConstrs(gp.quicksum(wijrp[i,j,r,p] for r in R)<=TRijp[i,j,p] for p in P for i in M+S for j in N)

m.addConstrs(wijrp[i,j,r,p]<=200* Yijrp[i,j,r,p] for p in range(0,len(P)) for i in M+S for j in N for r in R)


{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

In [54]:
TC=gp.quicksum(2*dM2TN[i][j][r]*Cost[r]*TRijp[i,j,p]for p in range(0,len(P)) for r in R for j in N for i in M+S)
MC=gp.quicksum(neta[j,f]*CF[j][f] for j in N for f in F)
HC=gp.quicksum(Vac_avai_jtp[j,t,p]-
               (gp.quicksum(XMijtp[i,j,t,p] for i in M)+
                gp.quicksum(XIijtp[i,j,t,p] for i in S)+
                gp.quicksum(gp.quicksum(xRMijtpz[i,j,t,p,z] for i in M)+
                gp.quicksum(xRIijtpz[i,j,t,p,z] for i in S) for z in range(1, Z+1)))for j in N for t in T for p in P)

CH=gp.quicksum(Cost[r]*Yijrp[i,j,r,p] for p in P for i in M+S for j in N for r in R)
UD=gp.quicksum(D_v_jtpz[j,t,p-1,z]-(gp.quicksum(xRMijtpz[i,j,t,p-1,z] for i in M)+gp.quicksum(xRIijtpz[i,j,t,p-1,z] for i in S)) for p in range(1,len(P)) for j in N for t in T for z in range(1, Z+1))
m.setObjective(TC+MC+HC+CH+UD, GRB.MINIMIZE)



In [55]:
m.Params.Cuts=0
m.Params.Heuristics=0
m.Params.Presolve=0
m.Params.Symmetry=0
m.Params.RINS=0
m.Params.Aggregate=0
m.Params.Disconnected=0
m.setParam("IntegralityFocus",1)
m.setParam("IntFeasTol",1e-09)
# m.setParam("MIPGap", 0.000000)
m.setParam("MIPGap", 0.00001)

m.optimize()

Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Presolve to value 0
Set parameter Symmetry to value 0
Set parameter RINS to value 0
Set parameter Aggregate to value 0
Set parameter Disconnected to value 0
Set parameter IntegralityFocus to value 1
Set parameter IntFeasTol to value 1e-09
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1460 rows, 1515 columns and 8075 nonzeros
Model fingerprint: 0x9b45945a
Variable types: 1100 continuous, 415 integer (315 binary)
Coefficient statistics:
  Matrix range     [2e-03, 4e+08]
  Objective range  [2e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avo

In [56]:
def extract_and_display_results(m, P):
    results_by_period = {p: [] for p in P}
    
    for var in m.getVars():
        var_name = var.VarName
        var_value = var.X
        if var_value > 0:  # Filter out variables with value <= 0
            try:
                parts = var_name.split('[')
                indices = parts[-1].split(',')
                if len(indices) == 4:
                    p = int(indices[-1][:-1])  # 'p' is the last index
                elif len(indices) == 5:
                    p = int(indices[-2])  # 'p' is the second-to-last index
                results_by_period[p].append((var_name, var_value))
            except Exception as e:
                print(f"Error parsing variable name '{var_name}': {e}")
    
    # Display results for all periods in P
    for p in P:
        if results_by_period[p]:
            print(f"Results for period p = {p}:")
            for var_name, var_value in results_by_period[p]:
                print(f"{var_name}: {var_value}")
            print("\n")
        else:
            print(f"No results for period p = {p}\n")


In [57]:
m.printAttr('X')


    Variable            X 
-------------------------
XMijtp[0,0,0,0]  3.13876e+07 
XMijtp[0,0,0,1]  3.13876e+07 
XMijtp[0,0,0,2]  1.19638e+07 
XMijtp[0,0,0,3]  1.49809e+07 
XMijtp[0,0,0,4]  1.40472e+07 
XMijtp[0,1,0,0]  5.33912e+07 
XMijtp[0,1,0,1]  5.33912e+07 
XMijtp[0,1,0,2]  1.60174e+07 
XMijtp[0,1,0,3]  1.60174e+07 
XMijtp[0,1,0,4]  1.60174e+07 
XMijtp[0,2,0,0]  3.32212e+07 
XMijtp[0,2,0,1]  3.32212e+07 
XMijtp[0,2,0,2]  9.96636e+06 
XMijtp[0,2,0,3]  9.96636e+06 
XMijtp[0,2,0,4]  9.96636e+06 
XMijtp[1,0,1,0]  1.65511e+07 
XMijtp[1,0,1,1]  1.65487e+07 
XMijtp[1,0,1,2]  2.07314e+06 
XMijtp[1,3,1,0]  9.25514e+07 
XMijtp[1,3,1,1]  9.25514e+07 
XMijtp[1,3,1,2]  2.77654e+07 
XMijtp[1,3,1,3]  2.77654e+07 
XMijtp[1,3,1,4]  2.77654e+07 
XMijtp[1,4,1,0]  8.89747e+06 
XMijtp[1,4,1,1]  8.89747e+06 
XMijtp[1,4,1,2]  2.66924e+06 
XMijtp[1,4,1,3]  2.66924e+06 
XMijtp[1,4,1,4]  2.66924e+06 
XIijtp[2,0,2,0]       997623 
XIijtp[2,0,2,1]        1e+06 
XIijtp[3,0,3,0]        1e+06 
XIijtp[3,0,3,1] 

In [58]:
extract_and_display_results(m, P)

Results for period p = 0:
XMijtp[0,0,0,0]: 31387643.0
XMijtp[0,1,0,0]: 53391170.0
XMijtp[0,2,0,0]: 33221187.0
XMijtp[1,0,1,0]: 16551086.0
XMijtp[1,3,1,0]: 92551442.0
XMijtp[1,4,1,0]: 8897472.0
XIijtp[2,0,2,0]: 997623.0
XIijtp[3,0,3,0]: 1000000.0
Yijrp[0,0,0,0]: 1.0
Yijrp[0,1,0,0]: 1.0
Yijrp[0,2,0,0]: 1.0
Yijrp[1,0,0,0]: 1.0
Yijrp[1,3,0,0]: 1.0
Yijrp[1,4,0,0]: 1.0
Yijrp[2,0,1,0]: 1.0
Yijrp[3,0,1,0]: 1.0
wijrp[0,0,0,0]: 1.0
wijrp[0,1,0,0]: 1.0
wijrp[0,2,0,0]: 1.0
wijrp[1,0,0,0]: 1.0
wijrp[1,3,0,0]: 1.0
wijrp[1,4,0,0]: 1.0
wijrp[2,0,1,0]: 1.0
wijrp[3,0,1,0]: 1.0


Results for period p = 1:
XMijtp[0,0,0,1]: 31387643.0
XMijtp[0,1,0,1]: 53391170.0
XMijtp[0,2,0,1]: 33221187.0
XMijtp[1,0,1,1]: 16548709.0
XMijtp[1,3,1,1]: 92551442.0
XMijtp[1,4,1,1]: 8897471.0
XIijtp[2,0,2,1]: 1000000.0
XIijtp[3,0,3,1]: 1000000.0
Yijrp[0,0,0,1]: 1.0
Yijrp[0,1,0,1]: 1.0
Yijrp[0,2,0,1]: 1.0
Yijrp[1,0,0,1]: 1.0
Yijrp[1,3,0,1]: 1.0
Yijrp[1,4,0,1]: 1.0
Yijrp[2,0,0,1]: 1.0
Yijrp[3,0,1,1]: 1.0
wijrp[0,0,0,1]: 1.0
wijrp